In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import roc_curve, auc
import pylab as pl
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
import warnings                              # Disable some warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


In [3]:
# Read the dataset as a Pandas dataframe in memory
dataset_wage = dataiku.Dataset("Model_Ready")
df = dataset_wage.get_dataframe(limit=100000)

In [4]:
# Get some simple descriptive statistics
pdu.audit(df)

,_a_variable,_b_data_type,_c_cardinality,_d_missings,_e_sample_values
0,District_Ciudad Lineal,int64,2,0,"[1, 0]"
1,District_Fuencarral,int64,2,0,"[0, 1]"
2,District_Hortaleza,int64,2,0,"[0, 1]"
3,District_Latina,int64,2,0,"[0, 1]"
4,District_Moncloa,int64,2,0,"[0, 1]"
5,District_Moratalaz,int64,2,0,"[0, 1]"
6,District_Puente Vallecas,int64,2,0,"[0, 1]"
7,District_Retiro,int64,2,0,"[0, 1]"
8,District_Salamanca,int64,2,0,"[0, 1]"
9,District_Arganzuela,int64,2,0,"[0, 1]"


In [5]:
# Stepwise function
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.1, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit(method='bfgs')
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included]))).fit(method='bfgs')
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [6]:
# Fills X (removing variables with missing values)
selected_fields=df.drop(labels=["Rent_GreaterThan_2000"],axis=1)
# Sets y
y = df['Rent_GreaterThan_2000']

result = stepwise_selection(selected_fields, y)

In [7]:
print('resulting features:')
print(result)

In [8]:
# Adds selected features
X=df
for item in df.columns:
    if item not in result:
        X=X.drop(labels=[item],axis=1)
X = sm.add_constant(X)

# statsmodels (no regularization)
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit_regularized()
print(result.summary2())

# print Exp(B) and confusion matrix
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['Lower', 'Upper', 'Odds Ratio']
print ("\nexp(B) & confidence intervals: ")
print (np.exp(conf))

print("\nConfusion matrix:")
print(result.pred_table())

In [9]:
train_cols = X.columns[1:]
result = sm.Logit(df['Rent_GreaterThan_2000'], df[train_cols]).fit()
df['pred'] = result.predict(df[train_cols])
fpr, tpr, thresholds =roc_curve(df['Rent_GreaterThan_2000'], df['pred'])
roc_auc = auc(fpr, tpr)
print("Area under the ROC curve : %f" % roc_auc)

i = np.arange(len(tpr)) # index for df
roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'cut-off' : pd.Series(thresholds, index=i)})
roc_t = roc.ix[(roc.tf-0).abs().argsort()[:1]]
print("Optimal cut-off for accuracy :")
print(list(roc_t['cut-off']))